# Air-Quality Sentinel – Exploratory Analysis

Inspect combined air-quality dataset to guide calibration, feature engineering, and spatial graph construction.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style='whitegrid')

PROJECT_ROOT = Path('..').resolve()
DATA_PATH = PROJECT_ROOT / 'data' / 'processed' / 'air_quality_combined.parquet'
STATIONS_META_PATH = PROJECT_ROOT / 'data' / 'interim' / 'stations_metadata.csv'

df = pd.read_parquet(DATA_PATH)
df.head()


In [ ]:
# Normalize timestamp and create calendar features
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=False)
df["date"] = df["timestamp"].dt.date
df["hour"] = df["timestamp"].dt.hour
df["dayofweek"] = df["timestamp"].dt.dayofweek
df["month"] = df["timestamp"].dt.month
df.describe(include='all').T.head(10)


In [ ]:
# Sensor coverage per source
coverage = df.groupby("source")["timestamp"].agg(['min','max','count']).sort_values("count", ascending=False)
coverage.head()

In [ ]:
# Example pollutant trends for the most frequent source
top_source = coverage.index[0]
pollutants = [col for col in df.columns if col in {"pm2.5", "pm10", "no2", "o3", "so2", "co"}]
subset = df[df["source"] == top_source].set_index("timestamp")

subset[pollutants].resample('D').mean().plot(figsize=(12,6))
plt.title(f"Daily Average Pollutants – {top_source}")
plt.ylabel("Concentration")
plt.show()

In [ ]:
# Missing value heatmap for pollutants
plt.figure(figsize=(10,5))
sns.heatmap(df[pollutants].isna().mean().to_frame(name="missing_ratio"), annot=True, cmap="Reds")
plt.title("Missing Value Ratios per Pollutant")
plt.show()

In [ ]:
# Merge station metadata if available for spatial context
if STATIONS_META_PATH.exists():
    stations_meta = pd.read_csv(STATIONS_META_PATH)
    stations_meta.columns = [c.lower() for c in stations_meta.columns]
    df = df.merge(stations_meta, how='left', on='stationid')
    print('Stations merged:', df['stationname'].nunique())
else:
    print('Station metadata not available; skipping merge.')


In [ ]:
# Seasonal patterns by month and hour for PM2.5
target_col = 'pm2.5' if 'pm2.5' in df.columns else 'pm2_5'
monthly = df.groupby('month')[target_col].mean()
hourly = df.groupby('hour')[target_col].mean()
fig, axes = plt.subplots(1, 2, figsize=(12,4))
sns.barplot(x=monthly.index, y=monthly.values, ax=axes[0], palette='Blues_d')
axes[0].set_title('Monthly Average PM2.5')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('µg/m³')
sns.barplot(x=hourly.index, y=hourly.values, ax=axes[1], palette='Greens_d')
axes[1].set_title('Hourly Average PM2.5')
axes[1].set_xlabel('Hour')
axes[1].set_ylabel('µg/m³')
plt.tight_layout()


In [ ]:
# Correlation matrix among pollutants (daily averages)
pollutant_cols = [col for col in df.columns if col in ['pm2.5', 'pm10', 'no', 'no2', 'nox', 'nh3', 'co', 'so2', 'o3']]
daily = df.set_index('timestamp')[pollutant_cols].resample('D').mean()
corr = daily.corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix (Daily Averages)')
plt.show()


In [ ]:
# Station-specific bias vs overall median
if 'stationid' in df.columns:
    station_counts = df['stationid'].value_counts()
    top_station = station_counts.index[0]
    station_series = df[df['stationid'] == top_station].set_index('timestamp')[target_col]
    overall_series = df.set_index('timestamp')[target_col]
    fig, ax = plt.subplots(figsize=(10,4))
    station_series.resample('W').mean().plot(ax=ax, label=f'Station {top_station}')
    overall_series.resample('W').median().plot(ax=ax, label='Overall Median', linestyle='--')
    ax.set_title('Weekly PM2.5 – Station vs Overall Median')
    ax.set_ylabel('µg/m³')
    ax.legend()
    plt.tight_layout()
else:
    print('Station IDs not available for bias comparison.')


In [ ]:
# Reference calibration: compare station readings with city-level median
if 'city' in df.columns:
    df['city'] = df['city'].fillna('Unknown')
    city_medians = df.groupby([df['timestamp'].dt.date, 'city'])[target_col].median().rename('city_median')
    df_city = df.copy()
    df_city['date'] = df_city['timestamp'].dt.date
    df_city = df_city.join(city_medians, on=['date', 'city'])
    df_city['calibration_error'] = df_city[target_col] - df_city['city_median']
    calibration_stats = df_city.groupby('stationid')['calibration_error'].agg(['mean','std','count']).sort_values('mean')
    calibration_stats.head()
else:
    print('City information not available; skipping calibration comparison.')


In [ ]:
# Feature engineering candidates: lagged values and rolling statistics
df_fe = df.set_index('timestamp')
lags = [1, 3, 6, 12, 24]
for lag in lags:
    df_fe[f'{target_col}_lag_{lag}'] = df_fe[target_col].shift(lag)
df_fe[f'{target_col}_rolling_mean_24'] = df_fe[target_col].rolling('24H').mean()
df_fe[f'{target_col}_rolling_std_24'] = df_fe[target_col].rolling('24H').std()
feature_summary = df_fe[[col for col in df_fe.columns if 'lag' in col or 'rolling' in col]].describe().T
feature_summary.head()


In [ ]:
# Visualize calibration error distribution for top stations
if 'calibration_error' in locals():
    top_stations = calibration_stats.index[:5]
    plt.figure(figsize=(10,5))
    sns.boxplot(data=df_city[df_city['stationid'].isin(top_stations)], x='stationid', y='calibration_error')
    plt.title('Calibration Error Distribution for Top Stations')
    plt.ylabel('PM2.5 - City Median (µg/m³)')
    plt.xlabel('Station ID')
    plt.tight_layout()
else:
    print('Calibration error not computed; skipping plot.')


## Next Steps
- Calibrate sensor readings using reference station comparisons (complete initial analysis).
- Integrate weather features (temperature, humidity, wind) for co-variates.
- Build spatial adjacency graph (k-NN by geolocation).
- Quantify uncertainty and define evaluation metrics (MAE, RMSE, MAPE, CRPS).
- Proceed to spatio-temporal modeling with TGAT/TFT architectures.
